R environment

In [ ]:
library(Seurat)
library(dplyr)
library(cowplot)
library(ggplot2)
library(copykat)
library(patchwork)
library(future)
source("modified_copykat.R")
plan("multiprocess", workers = 12)
options(future.globals.maxSize = 10000 * 1024^10,
        future.rng.onMisuse = 'ignore')
sessionInfo()

Attaching SeuratObject


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘patchwork’


The following object is masked from ‘package:cowplot’:

    align_plots




R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /hpc/pmc_stunnenberg/cruiz/miniconda3/envs/r_pHGG_project/lib/libopenblasp-r0.3.12.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] future_1.21.0      patchwork_1.1.1    copykat_1.0.4      ggplot2_3.3.3     
[5] cowplot_1.1.1      dplyr_1.0.6        SeuratObject_4.0.1 Seurat_4.0.1.9005 

loaded via a namespace (and not attached):
  [1] nlme_3.1-152          spatstat.sparse_2.0-0 matrixStats_0.58.0   
  [

In [ ]:
reference.list <- readRDS('data/core_GBmap_seuratobj_list.rds')
reference.list

$Yuan2018
An object of class Seurat 
28951 features across 12202 samples within 1 assay 
Active assay: RNA (28951 features, 0 variable features)

$Neftel2019_10x
An object of class Seurat 
22966 features across 13288 samples within 1 assay 
Active assay: RNA (22966 features, 0 variable features)

$Neftel2019_smart
An object of class Seurat 
21718 features across 5742 samples within 1 assay 
Active assay: RNA (21718 features, 0 variable features)

$Wang2019
An object of class Seurat 
38129 features across 23531 samples within 1 assay 
Active assay: RNA (38129 features, 0 variable features)

$Wang2020
An object of class Seurat 
17566 features across 12437 samples within 1 assay 
Active assay: RNA (17566 features, 0 variable features)

$Zhao2020
An object of class Seurat 
18774 features across 5182 samples within 1 assay 
Active assay: RNA (18774 features, 0 variable features)

$Bhaduri2020
An object of class Seurat 
22183 features across 14934 samples within 1 assay 
Active assay: RNA (2

In [ ]:
tmp <- merge(reference.list[[1]], y = unlist(reference.list[2:length(reference.list)]), project = "GBM", merge.data = T)

# Output a logical vector for every gene on whether the more than zero counts per cell
# Extract counts
counts <- GetAssayData(object = tmp, slot = "counts") 

# Output a logical vector for every gene on whether the more than zero counts per cell
nonzero <- counts > 0

# Sums all TRUE values and returns TRUE if more than 1000 TRUE values per gene
keep_genes <- Matrix::rowSums(nonzero) >= 100

# Only keeping those genes expressed in more than 1000 cells
filtered_counts <- counts[keep_genes, ]

# Read metadata
meta_all <- readRDS('data/core_GBmap_metadata.rds')
meta_all <- meta_all %>% mutate(celltype_assigned = read.csv('data/assigned_cellID_celltypes_core_GBmap.csv')$celltype_assigned)

# Reassign to filtered Seurat object
tmp <- CreateSeuratObject(filtered_counts, meta.data = meta_all)

In [ ]:
reference.list <- SplitObject(tmp, split.by = 'author')
# filter out datasets with only immune cells
non.immune <- reference.list[names(reference.list) %in% c('Sankowski2019','Goswami2019','Pombo2021','Mathewson2021') == FALSE] 
rm(reference.list)
rm(tmp)
non.immune

$Yuan2018
An object of class Seurat 
36498 features across 12202 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Neftel2019_10x
An object of class Seurat 
36498 features across 13288 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Neftel2019_smart
An object of class Seurat 
36498 features across 5742 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Wang2019
An object of class Seurat 
36498 features across 23531 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Wang2020
An object of class Seurat 
36498 features across 12437 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Zhao2020
An object of class Seurat 
36498 features across 5182 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Bhaduri2020
An object of class Seurat 
36498 features across 14934 samples within 1 assay 
Active assay: RNA (3

In [ ]:
# run first seven datsets
non.immune <- head(non.immune, n = 10)
non.immune

$Yuan2018
An object of class Seurat 
36498 features across 12202 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Neftel2019_10x
An object of class Seurat 
36498 features across 13288 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Neftel2019_smart
An object of class Seurat 
36498 features across 5742 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Wang2019
An object of class Seurat 
36498 features across 23531 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Wang2020
An object of class Seurat 
36498 features across 12437 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Zhao2020
An object of class Seurat 
36498 features across 5182 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$Bhaduri2020
An object of class Seurat 
36498 features across 14934 samples within 1 assay 
Active assay: RNA (3

In [ ]:
gbm <- merge(non.immune[[1]], y = unlist(non.immune[2:length(non.immune)]), project = "GBM", merge.data = T)
rm(non.immune)
gbm
per.sample <- SplitObject(gbm, split.by = 'patient')
rm(gbm)
head(per.sample)

An object of class Seurat 
36498 features across 167808 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$PJ017
An object of class Seurat 
36498 features across 944 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$PJ018
An object of class Seurat 
36498 features across 1942 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$PJ025
An object of class Seurat 
36498 features across 4549 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$PJ032
An object of class Seurat 
36498 features across 763 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$PJ035
An object of class Seurat 
36498 features across 1826 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)

$PJ048
An object of class Seurat 
36498 features across 2178 samples within 1 assay 
Active assay: RNA (36498 features, 0 variable features)


In [ ]:
copykat_dir <- "copykat_output_ref_normal_cells_part_I"

ifelse(!dir.exists(copykat_dir), dir.create(copykat_dir), FALSE)
setwd(copykat_dir)

for(i in 1:length(per.sample)){
    
    cat(' #####################################\n',
      '### Processing patient', names(per.sample[i]), '###\n',
      '#####################################\n')
    
    copykat.gbm <- try(modif_copykat(rawmat=GetAssayData(per.sample[[i]], slot = 'counts'), id.type="S", ngene.chr=1, 
                           win.size=25, KS.cut=0.1, sam.name=names(per.sample[i]), distance="euclidean",
                           norm.cell.names=colnames(subset(per.sample[[i]], subset = celltype_assigned == c('Oligo') |
                                 celltype_assigned == c('TAM') |
                                 celltype_assigned == c('TIL') |
                                 celltype_assigned == c('Peri') |
                                 celltype_assigned == c('Endo'))), 
                                     n.cores=12))
    
    if (class(copykat.gbm) == "try-error") {
                        cat("Caught an error during modif_copykat, Setting less stringent cutoff.\n")
                        copykat.gbm <- modif_copykat(rawmat=GetAssayData(per.sample[[i]], slot = 'counts'), id.type="S", ngene.chr=1, 
                           win.size=25, KS.cut=0.1, sam.name=names(per.sample[i]), distance="euclidean",LOW.DR=0.01,
                           norm.cell.names=colnames(subset(per.sample[[i]], subset = celltype_assigned == c('Oligo') |
                                 celltype_assigned == c('TAM') |
                                 celltype_assigned == c('TIL') |
                                 celltype_assigned == c('Peri') |
                                 celltype_assigned == c('Endo'))), 
                                                     n.cores=12)
        }
        
    pred.gbm <- data.frame(copykat.gbm$prediction)
    CNA.gbm <- data.frame(copykat.gbm$CNAmat)
    per.sample[[i]]@misc$CNA <- CNA.gbm
    pred.gbm['cell.names'] <- NULL
    per.sample[[i]]$predCNA <- pred.gbm
}

 #####################################
 ### Processing patient SF10022 ###
 #####################################
[1] "running copykat v1.0.4"
[1] "step1: read and filter data ..."
[1] "36498 genes, 2743 cells in raw data"
[1] "11927 genes past LOW.DR filtering"
[1] "step 2: annotations gene coordinates ..."
[1] "start annotation ..."
[1] "step 3: smoothing data with dlm ..."
[1] "step 4: measuring baselines ..."
[1] "389 known normal cells found in dataset"
[1] "run with known normal..."
[1] "baseline is from known input"
[1] "step 5: segmentation..."
[1] "step 6: convert to genomic bins..."
[1] "step 7: adjust baseline ..."
[1] "step 8: final prediction ..."
[1] "step 9: saving results..."
[1] "step 10: ploting heatmap ..."
Time difference of 17.09667 mins
 #####################################
 ### Processing patient SF10127 ###
 #####################################
[1] "running copykat v1.0.4"
[1] "step1: read and filter data ..."
[1] "36498 genes, 772 cells in raw data"
[1] "5596

In [ ]:
saveRDS(per.sample, 'data/copykat_ref_normal_cells_part_I.rds')